# LAB_01: Intro to FPGA designs!
So far you guys should have learned from PHYS_335 Advanced Digital Lab.:
 * Digital: 
   - gates, 
   - registers,
   - ALUs,
* Wait, no FSMs for now. Everyone interested in this area should get to know ASM/ASMD fairly quick later. This is a responsible practice.

In [2]:
SID = 1933453

# For this lab session: 
we will initially focus on fundamental FPGA projects to familiarize ourselves with:
* programmable logic,
* onboard GPIOs,
* and delve into the design process to gain an initial understanding of industry practices.

In [3]:
import os
import sympy as sp
# import numpy as np
import matplotlib.pyplot as plt

## TASK 1: LED Blink (TEST_IO)

You are currently working with the Zynq-7020 chip.

Before proceeding, ensure that the constraint files are inside the development file directory(where the project file is).

What this task should look like:
```systemverilog
module CHIP_7020_TOP (
   input logic [15:0] SW;
   output logic [0:0] LEDR[0];
);
   always_comb begin
        ...
        LEDR[0] = 1'b1; // LED blink
        ...
    end
endmodule
```

In [4]:
# AMD should provide this xdc file. If not, ask the guy who made this board.
# We can safely assume the IO names as standard when the below codes are being runned.
verilog = f"""
module CHIP_7020_TOP (
   input logic [15:0] SW;
   output logic [0:0] LEDR[0];
);
   always_comb begin
        LEDR[0] = 1'b1; // LED light up.
    endd
endmodule
"""
print(verilog)


module CHIP_7020_TOP (
   input logic [15:0] SW;
   output logic [0:0] LEDR[0];
);
   always_comb begin
        LEDR[0] = 1'b1; // LED light up.
    endd
endmodule



## TASK 2: LED Input Validation (TEST_LOGIC)

In this task, you will use the LED to indicate a specific condition based on input validation.

### Instructions:

1. **Understanding the Objective:**
   - The LED will be used to signify a desired condition once an input is validated correctly.

2. **Programming the FPGA:**
   - Begin by retrieving the last digit of your student number.
   - Implement a filter in the programmable logic to activate the LEDR only when your student ID is entered correctly using the switches.
     - This involves using Computational Boolean Algebra.
       1. Convert your last digit to binary representation.
       2. Program this binary filter logic onto the FPGA board.
       3. TAs will examine on other logics.

Core part of the task:
```systemverilog
module CHIP_7020_TOP (
   input logic [15:0] SW;
   output logic [0:0] LEDR[0];
);
   always_comb begin
      ...
      LEDR[0] = SID_BOOLEAN_FUNCTION(.in(SW[15:0]), .out(LEDR[0]));
      ...
   end
endmodule

module SID_BOOlLEAN_FUNCTION (
   input logic [15:0] in;
   output logic [0:0] out;
);
   always_comb begin
      out = // Your Student ID here. You can utilize an external logic library. We use Minterm in this task as basic pattern matcher.
   end
endmodule

```

### Key Steps:
- Identify your student number's last digit.
- Convert the last digit to binary.
- Develop and deploy the filter logic onto the FPGA to control LEDR based on valid input from the switches.

In [5]:
# Grab the last digit of your SID
SID_LAST_DIGIT = SID % 10
print(f"SID_LAST_DIGIT: {SID_LAST_DIGIT}")

# Generate the truth table for the SID_LAST_DIGIT, in this case the input upper bound is 9.
# SID_LAST_DIGIT_TRUTH_TABLE = [True if i == SID_LAST_DIGIT else False for i in range(2**np.ceil(np.log2(10)))] # Elementary way to see this.
# Got only 1 True in the truth table? Good. We got a Minterm.

# Let's do it in a more general way for minterms.
SID_LAST_DIGIT_BIN_STR = bin(SID_LAST_DIGIT)[2:]
# Sympy:
minterm = sp.true
for i in range(len(SID_LAST_DIGIT_BIN_STR)):
   toss_1 = SID_LAST_DIGIT_BIN_STR[i] == "1"
   minterm &= sp.Symbol(f"in[{i}]") if toss_1 else ~sp.Symbol(f"in[{i}]")
print("purely-CAS-engaged operation:")
print(f"Minterm: {minterm}")
# Programmatic:
minterm = ""
for i in range(len(SID_LAST_DIGIT_BIN_STR)):
   toss_1 = SID_LAST_DIGIT_BIN_STR[i] == "1"
   if i == len(bin(SID_LAST_DIGIT)[2:])-1:
      minterm += f"in[{i}]" if toss_1 else f"~in[{i}]"
   else:
      minterm += f"in[{i}] & " if toss_1 else f"~in[{i}] & "
print("General Programmatic operation:")
print(f"Minterm: {minterm}")

verilog = f"""
module CHIP_7020_TOP (
   input logic [15:0] SW;
   output logic [0:0] LEDR[0];
);
   always_comb begin
      LEDR[0] = SID_BOOLEAN_FUNCTION(.in(SW[15:0]), .out(LEDR[0]));
   end
endmodule

module SID_BOOlLEAN_FUNCTION (
   input logic [15:0] in;
   output logic [0:0] out;
);
   always_comb begin
      out = {minterm}// Your Student ID here. You can utilize an external logic library. We use Minterm in this task as basic pattern matcher.
   end
endmodule
"""
print(verilog)

SID_LAST_DIGIT: 3
purely-CAS-engaged operation:
Minterm: in[0] & in[1]
General Programmatic operation:
Minterm: in[0] & in[1]

module CHIP_7020_TOP (
   input logic [15:0] SW;
   output logic [0:0] LEDR[0];
);
   always_comb begin
      LEDR[0] = SID_BOOLEAN_FUNCTION(.in(SW[15:0]), .out(LEDR[0]));
   end
endmodule

module SID_BOOlLEAN_FUNCTION (
   input logic [15:0] in;
   output logic [0:0] out;
);
   always_comb begin
      out = in[0] & in[1]// Your Student ID here. You can utilize an external logic library. We use Minterm in this task as basic pattern matcher.
   end
endmodule



# Consult with Eason <- TODO

## TASK 3: Register Implementation (CURRENT_STATE_STORAGE)

You've now gained experience with logics. Let's proceed to implement state storage.

* State storage requires the concept of time, typically driven by a clock signal. This task focuses on utilizing time and implementing state storage using onboard registers.

### Instructions:

1. **Understanding State Storage:**
   - Implement registers to store and maintain state information.

```systemverilog
module CHIP_7020_TOP (
   input logic [15:0] SW;
   input logic clk; // Added
   output logic [0:0] LEDR[0];
);
   logic clk_divided, current_logic;
   reg[0:0] current_logic_reg;
   
   clock_divider(.in(clk), 
                 .out(clk_divided));
   
   SID_BOOLEAN_FUNCTION(.in(SW[15:0]), .out(current_logic));

   always_comb begin
      ...
      LEDR[0] = current_logic_reg;
      ...
   end

   always @(posedge clk_divided) begin // <- New content
      ... // Bunch of Transitions.
      current_logic_reg <= current_logic ? current_logic_reg ^ current_logic : 1'b0; // If we have valid switch inputs, the current_logic_reg will transit between 1 and 0. Otherwise, we make the value 0(not blinking and not even lighting up.)
      ...
   end
endmodule

module SID_BOOlLEAN_FUNCTION (
   input logic [15:0] in;
   output logic [0:0] out;
);
   always_comb begin
      out = // Your Student ID here. You can utilize an external logic library. We use Minterm in this task as basic pattern matcher.
   end
endmodule

// reg [n-1:0] reg_1; // n represents the bit width of the register
```

In [6]:
# Systematic way to do this:
class CHIP_TOP_MODULE:
   def __init__(self, IOs:dict[str:
                               dict[str:
                                    dict[str:
                                         tuple]]], 
                      FSMs:list[dict[str:
                                      dict[str: str]]], 
                      chip_name:str="CHIP_7020"):
      self.IOs = IOs
      self.FSMs = FSMs
      self.chip_name = chip_name

   def declare_IOs(self):
      IOs = ""
      for io_type in self.IOs.keys(): # digital, analog
         for io_direction in self.IOs[io_type].keys(): # input, output, inout
            for io_name in self.IOs[io_type][io_direction].keys(): # io_names, e.g. SW, LEDR
               IOs += f"   {io_direction} logic "
               for _range in self.IOs[io_type][io_direction][io_name]: # Basic range declaration, or multi-dimensional array declaration. We take care of multi-dimensional array declaration here.
                  if type(_range) == tuple:
                     IOs += f"[{_range[0]}:{_range[1]}]" # Range declaration.
                  else:
                     IOs += f"[{_range}:0]"
               IOs += f"{io_name};\n"
   
   def __str__(self):
      return f"""module {self.chip_name}_TOP (
      {self.declare_IOs()}
      );
      
      """
verilog = f"""
module CHIP_7020_TOP (
   input logic [15:0] SW;
   input logic clk;
   output logic [2:0] LEDR[2:0];
);
   logic clk_divided, current_logic;
   reg[0:0] current_logic_reg;
   
   clock_divider(.in(clk), 
                 .out(clk_divided));
   
   // SID_BOOLEAN_FUNCTION(.in(SW[15:0]), .out(current_logic));

   STD_INDICATOR_FSM(.SW_in(SW[15:0]),
                     .SID_Last_Digit_in((unsigned){len(SID_LAST_DIGIT_BIN_STR)}'b{SID_LAST_DIGIT_BIN_STR}),
                     .clk(clk_divided),
                     .out(LEDR[2:0]));

   // always_comb begin
   //    LEDR[0] = current_logic_reg;
   // end
   // 
   // always @(posedge clk_divided) begin
   //    ... // Bunch of Transitions.
   //    current_logic_reg <= current_logic ? current_logic_reg ^ current_logic : 1'b0; // If we have valid switch inputs, the current_logic_reg will transit between 1 and 0. Otherwise, we make the value 0(not blinking and not even lighting up.)
   //    ...
   // end
endmodule

module SID_BOOlLEAN_FUNCTION (
   input logic [15:0] in;
   output logic [0:0] out;
);
   always_comb begin
      out = (unsigned)in == (unsigned){len(SID_LAST_DIGIT_BIN_STR)}'b{SID_LAST_DIGIT_BIN_STR};// Your Student ID here. You can utilize an external logic library(recommended as a try). We use Minterm in this task as basic pattern matcher.
   end
endmodule
"""

# BONUS_TASK_4: (FSMs)
In this task, the FSM indicator should give correct pattern given the switch input is Greater, Smaller, or Equal to the SID number according to practices on cartitian coordinates R1:
* If greater, 3 LEDs will blink leftwards with each time one of them light up.
* If smaller, same thing but one of the LEDs light up rotating leftwards.
* If match, the output stops as center light up, then dim then the surrounding 2 lights up, repeat this until there is a match.
```Systemverilog
module CHIP_7020_TOP (
   input logic [15:0] SW;
   input logic clk;
   output logic [2:0] LEDR[2:0];
){
   logic clk_divided, current_logic;
   reg[0:0] current_logic_reg;
   
   clock_divider(.in(clk), 
                 .out(clk_divided));
   
   // SID_BOOLEAN_FUNCTION(.in(SW[15:0]), .out(current_logic));

   STD_INDICATOR_FSM(.SW_in(SW[15:0]), 
                     .SID_Last_Digit_in((unsigned)"YOUR_STUDENT_ID_HERE"), 
                     .clk(clk_divided), 
                     .out(LEDR[2:0]));

   // always_comb begin {
   //    ...
   //    LEDR[0] = current_logic_reg;
   //    ...
   // }
   // 
   // always @(posedge clk_divided) begin {
   //    ... // Bunch of Transitions.
   //    current_logic_reg <= current_logic ? current_logic_reg ^ current_logic : 1'b0; // If we have valid switch inputs, the current_logic_reg will transit between 1 and 0. Otherwise, we make the value 0(not blinking and not even lighting up.)
   //    ...
   // }
}

module STD_INDICATOR_FSM( // <- New Content
   input logic [15:0] SW_in;
   input logic [15:0] SID_Last_Digit_in;
   input logic clk;
   output logic [2:0] out;
) {
   logic [15:0] input_in_unsigned;
   // enum {state_G, state_L, state_E} curr_program_state, next_program_state;
   emum {light_LLED, light_MLED, light_RLED, light_LRLED} curr_light_state, next_light_state;
   logic [CEIL(LEN(curr_light_state)) - 1:0] next_light_state_result;

   light_nextstate_ROM(.*); 
   light_state_LEDR_lightup_ROM(.*);
   
   always_ff @(posedge_clk) begin: // Exhaustive on state space each sub LUT layer.
      next_light_state <= next_light_state_result;
   end
}

module light_nextstate_ROM(# NUM_STATES = 4)(
   input logic [15:0] SW_in;
   input logic [15:0] SID_Last_Digit_in;
   input logic [CEIL(LOG(NUM_STATES)) - 1:0] curr_light_state;
   output logic [CEIL(LOG(NUM_STATES)) - 1:0] next_light_state_result;
) {
   always_comb begin
      next_light_state_result = (curr_light_state == light_LRLED) ? ((unsigned)SW_in == (unsigned)SID_Last_Digit_in) ? light_MLED:
                                                              ((unsigned)SW_in > (unsigned)SID_Last_Digit_in) ? light_MLED:
                                                              ((unsigned)SW_in < (unsigned)SID_Last_Digit_in) ? light_MLED:
                                                            :
                          (curr_light_state == light_LLED) ? ((unsigned)SW_in == (unsigned)SID_Last_Digit_in) ? light_MLED:
                                                              ((unsigned)SW_in > (unsigned)SID_Last_Digit_in) ? light_RLED:
                                                              ((unsigned)SW_in < (unsigned)SID_Last_Digit_in) ? light_MLED:
                                                            :
                          (curr_light_state == light_MLED) ? ((unsigned)SW_in == (unsigned)SID_Last_Digit_in) ? light_LRLED:
                                                              ((unsigned)SW_in > (unsigned)SID_Last_Digit_in) ? light_LLED:
                                                              ((unsigned)SW_in < (unsigned)SID_Last_Digit_in) ? light_RLED:
                                                            :
                          (curr_light_state == light_RLED) ? ((unsigned)SW_in == (unsigned)SID_Last_Digit_in) ? light_MLED:
                                                              ((unsigned)SW_in > (unsigned)SID_Last_Digit_in) ? light_MLED:
                                                              ((unsigned)SW_in < (unsigned)SID_Last_Digit_in) ? light_LLED
                           ;
   end
}

module light_state_LEDR_lightup_ROM () (
   input logic [CEIL(LOG(NUM_STATES)) - 1:0] curr_light_state;
   output logic [2:0] out;
) {
   always_comb begin
      out = (curr_light_state == light_LLED) ? (out == 3'b100):
            (curr_light_state == light_MLED) ? (out == 3'b010):
            (curr_light_state == light_RLED) ? (out == 3'b001):
            (curr_light_state == light_LRLED) ? (out == 3'b101):
            3'b000; // safe measure
   end
}

// module SID_BOOlLEAN_FUNCTION (
//    input logic [15:0] in;
//    output logic [0:0] out;
// ) {
//    always_comb begin {
//       out = // Your Student ID here. You can utilize an external logic library. We use Minterm in this task as basic pattern matcher.
//    }
// }

// reg [n-1:0] reg_1; // n represents the bit width of the register
```

In [7]:
verilog = f"""
module CHIP_7020_TOP (
   input logic [15:0] SW;
   input logic clk;
   output logic [2:0] LEDR[2:0];
);
   logic clk_divided, current_logic;
   reg[0:0] current_logic_reg;
   
   clock_divider(.in(clk), 
                 .out(clk_divided));
   
   // SID_BOOLEAN_FUNCTION(.in(SW[15:0]), .out(current_logic));

   STD_INDICATOR_FSM(.SW_in(SW[15:0]),
                     .SID_Last_Digit_in((unsigned){len(SID_LAST_DIGIT_BIN_STR)}'b{SID_LAST_DIGIT_BIN_STR}),
                     .clk(clk_divided),
                     .out(LEDR[2:0]));

   // always_comb begin
   //    ...
   //    LEDR[0] = current_logic_reg;
   //    ...
   // end
   // 
   // always @(posedge clk_divided) begin
   //    ... // Bunch of Transitions.
   //    current_logic_reg <= current_logic ? current_logic_reg ^ current_logic : 1'b0; // If we have valid switch inputs, the current_logic_reg will transit between 1 and 0. Otherwise, we make the value 0(not blinking and not even lighting up.)
   //    ...
   // end
endmodule

module STD_INDICATOR_FSM( // <- New Content
   input logic [15:0] SW_in;
   input logic [15:0] SID_Last_Digit_in;
   input logic clk;
   output logic [2:0] out;
);
   logic [15:0] input_in_unsigned;
   // enum {{state_G, state_L, state_Eend curr_program_state, next_program_state}};
   emum {{light_LLED, light_MLED, light_RLED, light_LRLEDend curr_light_state, next_light_state}};
   logic [CEIL(LEN(curr_light_state)) - 1:0] next_light_state_result;

   light_nextstate_ROM(.*); 
   light_state_LEDR_lightup_ROM(.*);
   
   always_ff @(posedge_clk) begin: // Exhaustive on state space each sub LUT layer.
      next_light_state <= next_light_state_result;
   end
endmodule


module light_nextstate_ROM(# NUM_STATES = 4)(
   input logic [15:0] SW_in;
   input logic [15:0] SID_Last_Digit_in;
   input logic [CEIL(LOG(NUM_STATES)) - 1:0] curr_light_state;
   output logic [CEIL(LOG(NUM_STATES)) - 1:0] next_light_state_result;
);
   always_comb begin
      next_light_state_result = (curr_light_state == light_LRLED) ? ((unsigned)SW_in == (unsigned)SID_Last_Digit_in) ? light_MLED:
                                                              ((unsigned)SW_in > (unsigned)SID_Last_Digit_in) ? light_MLED:
                                                              ((unsigned)SW_in < (unsigned)SID_Last_Digit_in) ? light_MLED:
                                                            :
                                 (curr_light_state == light_LLED) ? ((unsigned)SW_in == (unsigned)SID_Last_Digit_in) ? light_MLED:
                                                                     ((unsigned)SW_in > (unsigned)SID_Last_Digit_in) ? light_RLED:
                                                                     ((unsigned)SW_in < (unsigned)SID_Last_Digit_in) ? light_MLED:
                                                                     :
                                 (curr_light_state == light_MLED) ? ((unsigned)SW_in == (unsigned)SID_Last_Digit_in) ? light_LRLED:
                                                                     ((unsigned)SW_in > (unsigned)SID_Last_Digit_in) ? light_LLED:
                                                                     ((unsigned)SW_in < (unsigned)SID_Last_Digit_in) ? light_RLED:
                                                                     :
                                 (curr_light_state == light_RLED) ? ((unsigned)SW_in == (unsigned)SID_Last_Digit_in) ? light_MLED:
                                                                     ((unsigned)SW_in > (unsigned)SID_Last_Digit_in) ? light_MLED:
                                                                     ((unsigned)SW_in < (unsigned)SID_Last_Digit_in) ? light_LLED
                                    ;
   end
endmodule


module light_state_LEDR_lightup_ROM () (
   input logic [CEIL(LOG(NUM_STATES)) - 1:0] curr_light_state;
   output logic [2:0] out;
);
   always_comb begin
      out = (curr_light_state == light_LLED) ? (out == 3'b100):
            (curr_light_state == light_MLED) ? (out == 3'b010):
            (curr_light_state == light_RLED) ? (out == 3'b001):
            (curr_light_state == light_LRLED) ? (out == 3'b101):
            3'b000; // safe measure
   end
endmodule

// module SID_BOOlLEAN_FUNCTION (
//    input logic [15:0] in;
//    output logic [0:0] out;
// );
//    always_comb begin
//       out = (unsigned)in == (unsigned){len(SID_LAST_DIGIT_BIN_STR)}'b{SID_LAST_DIGIT_BIN_STR};// Your Student ID here. You can utilize an external logic library(recommended as a try). We use Minterm in this task as basic pattern matcher.
//    end
// endmodule
"""

print(verilog)
os.makedirs("labworx_0", exist_ok=True)
os.chdir("labworx_0")
with open("7020_top.sv", "w") as f:
    f.write(verilog)
os.chdir("..")


module CHIP_7020_TOP (
   input logic [15:0] SW;
   input logic clk;
   output logic [2:0] LEDR[2:0];
);
   logic clk_divided, current_logic;
   reg[0:0] current_logic_reg;
   
   clock_divider(.in(clk), 
                 .out(clk_divided));
   
   // SID_BOOLEAN_FUNCTION(.in(SW[15:0]), .out(current_logic));

   STD_INDICATOR_FSM(.SW_in(SW[15:0]),
                     .SID_Last_Digit_in((unsigned)2'b11),
                     .clk(clk_divided),
                     .out(LEDR[2:0]));

   // always_comb begin
   //    ...
   //    LEDR[0] = current_logic_reg;
   //    ...
   // end
   // 
   // always @(posedge clk_divided) begin
   //    ... // Bunch of Transitions.
   //    current_logic_reg <= current_logic ? current_logic_reg ^ current_logic : 1'b0; // If we have valid switch inputs, the current_logic_reg will transit between 1 and 0. Otherwise, we make the value 0(not blinking and not even lighting up.)
   //    ...
   // end
endmodule

module STD_INDICATOR_FSM( // <- New Content
  

# What makes an ASM?
Algorithmetic state machine.

# Now, ASMDs.
Basically, it's an ASM with datapath.

# BONUS_TASK_5: (ASMs)